R for maps

https://www.emilyburchfield.org/courses/eds/making_maps_in_r

https://eriqande.github.io/rep-res-web/lectures/making-maps-with-R.html

Libraries

In [ ]:
install.packages(c("ggplot2","devtools","dplyr","stringr","maps","mapdata","ggmap"))
library(tidyverse)
library(sf)
library(tmap)
library(spData)
library(ggplot2)
library(ggmao)
library(maps)
library(mapdata)

Example: US Counties, Georgia

Major US Roads: 
https://catalog.data.gov/dataset/tiger-line-shapefile-2016-nation-u-s-primary-roads-national-shapefile
GA counties: 
https://arc-garc.opendata.arcgis.com/datasets/dc20713282734a73abe990995de40497_68


Read data, filter by name

In [ ]:
data("us_states") # from spData package
ga <- us_states %% filter(NAME == "Georgia")
counties <- st_read("myfolder/counties.shp", quiet=T)
roads <- st_read("myfolder/roads.shp")

Create projection variable

In [ ]:
proj <- "+proj=lcc +lat_1=31.41666666666667 +lat_2=34.28333333333333 +lat_0=0 +lon_0=-83.5 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +to_meter=0.3048006096012192 +no_defs"

Reproject inputs

In [ ]:
roads <- st_transform(roads, proj)
ga <- st_transform(ga, proj)
counties <- st_transform(counties, proj)

Clip road by selected state input

In [ ]:
roads <- st_crop(roads, ga)

Plot state, counties, and roads

In [ ]:
plot(st_geometry(ga))
plot(st_geometry(counties), add = T, col = "red")
plot(st_geometry(roads), add = T, col = "blue")

------------

Using ggplot

In [ ]:
ggplot(ga) +  geom_sf()

with title

In [ ]:
ggplot(ga) + 
  geom_sf(fill = "beige") +  
  labs(title = "The fine state of Georgia") +  
  theme_minimal()

with more info

In [ ]:
ggplot() +
  geom_sf(data = counties, aes(fill = totpop10)) +
  geom_sf(data = roads, color = "orange") +
  labs(title = "All roads lead to the ATL", fill = "Population") +
  theme_minimal()

Using ggspatial, north arrow, scalebar

In [ ]:
library(ggspatial)
ggplot() +
  geom_sf(data = counties, aes(fill = totpop10)) +
  geom_sf(data = roads, color = "orange") +
  labs(title = "All roads lead to the ATL", fill = "Population") +
  theme_minimal() +
  annotation_scale(location = "bl", width_hint = 0.5) +
  annotation_north_arrow(location = "tr", which_north = "true", style = north_arrow_fancy_orienteering())

Using tmap

In [ ]:
library(tmap)
tmap_mode("plot") 
qtm(roads)

choropleth quick thematic map

In [ ]:
qtm(counties, fill = "totpop10", fill.title = "Total population")

using tm shape

In [ ]:
tm_shape(counties) +
    tm_fill(col = "totpop10", convert2density = T, style = "jenks", 
    title = "Population (x100)") +
    tm_borders(alpha = 0.3) +
    tm_compass() +
    tm_scale_bar()

Save map

In [ ]:
save_tmap(tm, "./fig/my_awesome_map.png", width = 800, height = 1000)

View interactive map

In [ ]:
tmap_mode("view")
tm

---------

examples from 2nd link

we get a USA map from maps:

In [ ]:
usa <- map_data("usa")

dim(usa)

high-res world map centered on the Pacific Ocean from mapdata

In [ ]:
w2hr <- map_data("world2Hires")

dim(w2hr)

geom_polygon() draws with no line color, but with a black fill:

In [ ]:
ggplot() + geom_polygon(data = usa, aes(x=long, y = lat, group = group)) + 
  coord_fixed(1.3)

coord_fixed() fixes the relationship between one unit in the y direction and one unit in the x direction

No fill only outline:

In [ ]:
ggplot() + 
  geom_polygon(data = usa, aes(x=long, y = lat, group = group), fill = NA, color = "red") + 
  coord_fixed(1.3)

violet fill, with a blue line:

In [ ]:
gg1 <-ggplot() + 
  geom_polygon(data = usa, aes(x=long, y = lat, group = group), fill = "violet", color = "blue") + 
  coord_fixed(1.3)
gg1

Adding points to the map

In [ ]:
pts <- data.frame(
                  long = c(-122.064873, -122.306417),
                  lat = c(36.951968, 47.644855),
                  names = c("ptA", "ptB"),
                  stringsAsFactors = FALSE
                  )  

gg1 + 
  geom_point(data = pts, aes(x = long, y = lat), color = "black", size = 5) +
  geom_point(data = pts, aes(x = long, y = lat), color = "yellow", size = 4)

State maps

In [ ]:
states <- map_data("state")
dim(states)

Plot all the states, all colored a little differently

In [ ]:
ggplot(data = states) + 
  geom_polygon(aes(x = long, y = lat, fill = region, group = group), color = "white") + 
  coord_fixed(1.3) +
  guides(fill=FALSE)

counties

Get California, counties

In [ ]:
ca_df <- subset(states, region == "california")
counties <- map_data("county")
ca_county <- subset(counties, region == "california")

Plot state

In [ ]:
ca_base <- ggplot(data = ca_df, 
                  mapping = aes(x = long, y = lat, group = group)) + 
          coord_fixed(1.3) + 
          geom_polygon(color = "black", fill = "gray")
ca_base + theme_nothing()

plot the county boundaries in white

In [ ]:
ca_base + theme_nothing() + 
  geom_polygon(data = ca_county, fill = NA, color = "white") +
  geom_polygon(color = "black", fill = NA)  # get the state border back on top

stats about counties using regex

In [ ]:
library(stringr)
library(dplyr)

In [ ]:
 # make a data frame
    x <- readLines("data/ca-counties-wikipedia.txt")
    pop_and_area <- str_match(x, "^([a-zA-Z ]+)County\t.*\t([0-9,]{2,10})\t([0-9,]{2,10}) sq mi$")[, -1] %>%
      na.omit() %>%
      str_replace_all(",", "") %>% 
      str_trim() %>%
      tolower() %>%
      as.data.frame(stringsAsFactors = FALSE)

In [ ]:
 # give names and make population and area numeric
    names(pop_and_area) <- c("subregion", "population", "area")
    pop_and_area$population <- as.numeric(pop_and_area$population)
    pop_and_area$area <- as.numeric(pop_and_area$area)

In [ ]:
head(pop_and_area)

We now have the numbers that we want, but we need to attach those to every point on polygons of the counties. This is a job for inner_join from the dplyr package

In [ ]:
cacopa <- inner_join(ca_county, pop_and_area, by = "subregion")

add a column of people_per_mile

In [ ]:
cacopa$people_per_mile <- cacopa$population / cacopa$area

plot population density by county

In [ ]:
# prepare to drop the axes and ticks but leave the guides and legends

ditch_the_axes <- theme(
  axis.text = element_blank(),
  axis.line = element_blank(),
  axis.ticks = element_blank(),
  panel.border = element_blank(),
  panel.grid = element_blank(),
  axis.title = element_blank()
  )

elbow_room1 <- ca_base + 
      geom_polygon(data = cacopa, aes(fill = people_per_mile), color = "white") +
      geom_polygon(color = "black", fill = NA) +
      theme_bw() +
      ditch_the_axes

elbow_room1 


gradient color by pop density

In [ ]:
elbow_room1 + scale_fill_gradient(trans = "log10")

Different color ramp

In [ ]:
eb2 <- elbow_room1 + 
    scale_fill_gradientn(colours = rev(rainbow(7)),
                         breaks = c(2, 4, 10, 100, 1000, 10000),
                         trans = "log10")
eb2

Zoom in

In [ ]:
eb2 + xlim(-123, -121.0) + ylim(36, 38)

In [ ]:
eb2 + coord_fixed(xlim = c(-123, -121.0),  ylim = c(36, 38), ratio = 1.3)

----------------

Plot a bike route

In [ ]:
bike <- read.csv("data/bike-ride.csv")
head(bike)

Map from URL : http://maps.googleapis.com/maps/api/staticmap?center=36.971709,-122.080954&zoom=14&size=640x640&scale=2&maptype=terrain&language=en-EN&sensor=false

In [ ]:
bikemap1 <- get_map(location = c(-122.080954, 36.971709), maptype = "terrain", source = "google", zoom = 14)

plot

In [ ]:
ggmap(bikemap1) + 
  geom_path(data = bike, aes(color = elevation), size = 3, lineend = "round") + 
  scale_color_gradientn(colours = rainbow(7), breaks = seq(25, 200, by = 25))
